# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Weather_data (CSV)
weather_data = pd.read_csv("Weather_data.csv")
weather_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Marsala,40,IT,1570334092,78,37.80,12.44,71.60,14.99
1,1,Hobart,75,AU,1570333914,82,-42.88,147.33,62.01,13.87
2,2,Takefu,75,JP,1570334094,64,35.90,136.17,69.80,12.75
3,3,Abu Zabad,100,SD,1570334096,79,12.35,29.25,73.01,7.96
4,4,Vestmannaeyjar,66,IS,1570334097,86,63.44,-20.27,50.10,18.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
hum_weight = weather_data["Humidity"]

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=hum_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Drop rows that do not meet the criteria:
# Max Temp < 80 but >70
# Wind Speed < 10
#Cloudiness = 0
narrowed_city_df = weather_data.loc[((weather_data['Max Temp'] <= 80) & (weather_data['Max Temp'] >= 70)
                                                 & (weather_data['Wind Speed'] < 10) &
                                                 (weather_data['Cloudiness'] == 0))]
narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
91,91,Buraydah,0,SA,1570334152,41,26.33,43.97,77.00,4.70
123,123,Lar,0,IR,1570334253,94,27.66,54.32,71.60,6.93
151,151,Unai,0,BR,1570334194,57,-16.36,-46.91,72.07,1.30
167,167,Mecca,0,SA,1570334185,79,21.43,39.83,77.47,2.26
195,195,Cururupu,0,BR,1570334133,95,-1.82,-44.87,72.61,2.77
226,226,Floriano,0,BR,1570334387,53,-6.77,-43.02,76.18,2.01
280,280,Mogadishu,0,SO,1570334230,89,2.04,45.34,76.75,7.45
313,313,Wattegama,0,LK,1570334510,74,7.35,80.68,79.74,1.88
343,343,Kolokani,0,ML,1570334552,73,13.58,-8.03,75.85,4.32
369,369,Taoudenni,0,ML,1570334586,15,22.68,-3.98,79.22,6.89


In [6]:
# First filter out those rows which 
# does not contain any data 
hotel_df = narrowed_city_df.dropna(how = 'all')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Add a "Hotel Name" column to the DataFrame
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop=True)

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,91,Buraydah,0,SA,1570334152,41,26.33,43.97,77.00,4.70,
1,123,Lar,0,IR,1570334253,94,27.66,54.32,71.60,6.93,
2,151,Unai,0,BR,1570334194,57,-16.36,-46.91,72.07,1.30,
3,167,Mecca,0,SA,1570334185,79,21.43,39.83,77.47,2.26,
4,195,Cururupu,0,BR,1570334133,95,-1.82,-44.87,72.61,2.77,
5,226,Floriano,0,BR,1570334387,53,-6.77,-43.02,76.18,2.01,
6,280,Mogadishu,0,SO,1570334230,89,2.04,45.34,76.75,7.45,
7,313,Wattegama,0,LK,1570334510,74,7.35,80.68,79.74,1.88,
8,343,Kolokani,0,ML,1570334552,73,13.58,-8.03,75.85,4.32,
9,369,Taoudenni,0,ML,1570334586,15,22.68,-3.98,79.22,6.89,


In [8]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
#    "rankby": "distance",
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [9]:
# Showing results
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
91,91,Buraydah,0,SA,1570334152,41,26.33,43.97,77.00,4.70,Buraydah
123,123,Lar,0,IR,1570334253,94,27.66,54.32,71.60,6.93,Lar
151,151,Unai,0,BR,1570334194,57,-16.36,-46.91,72.07,1.30,Unaí
167,167,Mecca,0,SA,1570334185,79,21.43,39.83,77.47,2.26,Mecca
195,195,Cururupu,0,BR,1570334133,95,-1.82,-44.87,72.61,2.77,Cururupu
226,226,Floriano,0,BR,1570334387,53,-6.77,-43.02,76.18,2.01,Floriano
280,280,Mogadishu,0,SO,1570334230,89,2.04,45.34,76.75,7.45,Mogadishu
313,313,Wattegama,0,LK,1570334510,74,7.35,80.68,79.74,1.88,Kandy
343,343,Kolokani,0,ML,1570334552,73,13.58,-8.03,75.85,4.32,Kolokani
369,369,Taoudenni,0,ML,1570334586,15,22.68,-3.98,79.22,6.89,Taoudenni


In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
coordinates = hotel_df[["Lat", "Lng"]]


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))